In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from files.Simulation import Simulation
from files.Source import Source
from files.Observer import Observer
from plot.Trajectory import Trajectory

## Simulation setup

#### Parameters

In [46]:
nr_particles = 3*10**2
nr_steps = 3*10**4
source_pos = np.array([0.0, 0.0, 0.0], dtype=np.float32)
observer_substeps = np.array([False, False, True]) # observe only steps (no substeps)
mfp = [3.75*10**13/4.0, 3.75*10**13/4.0, 7.2*10**13]  # [m]
delta_rho_div_phi = 1 # (delta_r_rho / delta_r_phi)
gyro_radius = 10**11 # [m]; 1pc
all_steps = False

In [47]:
sim = Simulation()

source = Source(gyro_radius, mfp, source_pos, nr_particles)
sim.add_source(source)

observer = Observer(observer_substeps, all_steps)
sim.add_observer(observer)

%time sim.run_simulation(nr_steps)
sim.save_data('data')

start simulation
source
observer
CPU times: user 25.7 s, sys: 220 ms, total: 25.9 s
Wall time: 25.9 s


## Analyze statistics


In [106]:
class Statistics():
    def __init__(self, df, dimensions):
        print('init statistics plotting class')
        self.df = df
        self.dimensions = dimensions
    
    def plot_distribution(self, axis, step, bins, file_name):
        df_i = self.df[self.df['i'] == step]
        plt.figure(figsize=(4,4))
        plt.hist(df_i[axis], bins)
        plt.xlabel(axis + ' [m]')
        plt.ylabel('# particles')
        #plt.legend()
        if file_name is not None:
            plt.tight_layout()
            plt.savefig(file_name)
        plt.show()
        
    def plot_diffusion_coefficients(self):
        nr_particle = len(list(map(int, (set(self.df['id'])))))
        df = self.df.sort_values('d')
        x = df['x'].values[1:]
        y = df['y'].values[1:]
        z = df['z'].values[1:]
        t = df['d'].values[1:]
        print(t[:100])
        times = []
        kappa_xx = []
        kappa_yy = []
        kappa_perp = []
        kappa_zz = []
        for j in range(int(len(x)/nr_particles)):
            t_j = t[j*nr_particles]
            kappa_xx_current = 0
            kappa_yy_current = 0
            kappa_zz_current = 0
            for i in range(nr_particles):
                x_i = x[j*nr_particles+i]
                y_i = y[j*nr_particles+i]
                z_i = z[j*nr_particles+i]
                kappa_xx_current = kappa_xx_current + x_i**2
                kappa_yy_current = kappa_yy_current + y_i**2
                kappa_zz_current = kappa_zz_current + z_i**2
            kappa_xx.append(kappa_xx_current/(2*t_j))
            kappa_yy.append(kappa_yy_current/(2*t_j))
            kappa_perp.append((kappa_xx_current+kappa_yy_current)/(4*t_j))
            kappa_zz.append(kappa_zz_current/(2*t_j))
            times.append(t_j)
        plt.figure(figsize=(4,4))
        plt.plot(times, kappa_xx, label='$\kappa_{xx}$')
        plt.plot(times, kappa_yy, label='$\kappa_{yy}$')
        plt.plot(times, kappa_perp, label='$\kappa_\perp$')
        plt.plot(times, kappa_zz, label='$\kappa_\parallel$')
        plt.loglog()
        plt.legend()
        plt.show()
        
        d = {'kappa_perp':kappa_perp,'kappa_para':kappa_zz}
        return pd.DataFrame(d)
           

In [107]:
#from plot.Trajectory import Trajectory

df = pd.read_pickle("data.pkl")
df_time_evolution_observer = df.loc[df['radius'] == -1.0]
dimensions = 3
sta = Statistics(df_time_evolution_observer, dimensions)
tra = Trajectory(df_time_evolution_observer, dimensions)
particle_ids = tra.get_particle_ids()

init statistics plotting class
init trajectory plotting class


In [ ]:
nr_steps = 3*10**4
tra.plot_trajectory('x', 'y', 'd', particle_ids[0], nr_steps, None)

In [ ]:
bins = 20
file_name = None
sta.plot_distribution('x', 9990, bins, file_name)

In [ ]:
df_kappas = sta.plot_diffusion_coefficients()

In [105]:
df_kappas

,kappa_perp,kappa_para
0,inf,inf
1,2.588103e+11,2.525000e+11
2,5.143099e+11,5.020833e+11
3,7.694526e+11,7.519444e+11
4,1.023706e+12,1.001875e+12
...,...,...
3174,1.133075e+11,4.145733e+15
3175,1.101045e+11,4.123770e+15
3176,1.100855e+11,4.121090e+15
3177,1.085953e+11,4.153059e+15
